File used to test the accuracy of inverse solution of the motion model used in reconstruction (scripts\catheter_reconstruction\catheter_motion_tensor.py, scripts\catheter_reconstruction\bezier_interspace_transforms_tensor.py) 

In [1]:
import numpy as np
import torch

from bezier_interspace_transforms_tensor import tendon_disp_to_bezier_control_points, bezier_control_points_to_tendon_disp
from catheter_motion_tensor import CatheterMotion

In [2]:
if torch.cuda.is_available():
        gpu_or_cpu = torch.device("cuda:0") 
        torch.cuda.set_device(gpu_or_cpu)
else:
    gpu_or_cpu = torch.device("cpu")

p0 = torch.tensor([0.02, 0.002, 0.000001]).to(gpu_or_cpu)
para_init = torch.tensor([0.034, -0.01, 0.536, 0.2, -0.37, 0.6]).to(gpu_or_cpu)
delta_u_list = torch.tensor([[0, -0.0002], [0, -0.0002], [0, -0.0002], [0, -0.0002]]).to(gpu_or_cpu)

catheterMotion = CatheterMotion(p0, gpu_or_cpu, l=0.2, r=0.01)
predicted_paras = catheterMotion.past_frames_prediction(delta_u_list, para_init)

In [3]:
p_0_h = torch.tensor([0.02, 0.002, 0.000001, 1]).to(gpu_or_cpu)
l = 0.2
r = 0.01
ux = torch.tensor(0.003).to(gpu_or_cpu)
uy = torch.tensor(0.002).to(gpu_or_cpu)

In [4]:
p1, p2 = tendon_disp_to_bezier_control_points(ux, uy, l, r, p_0_h, gpu_or_cpu)
ux_solve, uy_solve = bezier_control_points_to_tendon_disp(p_0_h, p1, p2, l, r, gpu_or_cpu)

ux_solve = ux_solve.item()
uy_solve = uy_solve.item()
print(ux_solve, uy_solve)

ux_error = abs((ux_solve - ux) / ux) * 100 if ux != 0 else 0
uy_error = abs((uy_solve - uy) / uy) * 100 if uy != 0 else 0
print(f"ux: {ux:.4f}, uy: {uy:.4f} -> ux_solve: {ux_solve:.4f}, uy_solve: {uy_solve:.4f}, ux_error: {ux_error:.2f}%, uy_error: {uy_error:.2f}%")

0.002906977664679289 0.0019379849545657635
ux: 0.0030, uy: 0.0020 -> ux_solve: 0.0029, uy_solve: 0.0019, ux_error: 3.10%, uy_error: 3.10%
